# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html
- Important note! DeepState, DF-RNN are only released for versions >="0.4.*"
- For development version, follow **Install from Source Code** (which I did) >> **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [19]:
!pip install mxnet=="1.4.1"

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
!pip install gluonts

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import gluonts
gluonts.__version__

'0.4.1'

In [7]:
import mxnet
mxnet.__version__

'1.4.1'

In [3]:
import pprint

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [4]:
def deep_state(seed=42, data="m4_quarterly", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)

    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,

    )

    predictor = estimator.train(dataset.train)
    
#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    
    df=pd.DataFrame([output])
    pprint(output)
    
    return(df)

In [10]:
%%time
if __name__ == "__main__":
    df = deep_state(seed=42, data="m4_weekly", epochs=2, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [14:20<00:00,  8.60s/it, avg_epoch_loss=-.182] 
INFO:root:Epoch[0] Elapsed time 860.270 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-0.182407


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'C:\\Users\\TM\\AppData\\Local\\Temp\\gluonts-trainer-temp-m1eqbwmc'

In [11]:
df

NameError: name 'df' is not defined

In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html deep_state01-deep_state_experiments_with_gluonts.ipynb

[NbConvertApp] Converting notebook DeepState01-DeepState_experiments_with_gluonts.ipynb to html
[NbConvertApp] Writing 1312413 bytes to ./html_outputs/DeepState01-DeepState_experiments_with_gluonts.html


In [7]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deep_state(data="m4_weekly", seed=i, epochs=1, batches=1)
#         pprint(res)
#         results.append(res)
    
# results